## Setup

In [50]:
import tensorflow as tf
from tensorflow import keras
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, UnidentifiedImageError
import os
from random import randint

## Resize images to be the same size

In [51]:
target_size = (64, 64)

# Resize all images to be target_size.
# First, resize so that the shortest side is the same as target_size,
#   then crop with the center as the anchor.
for root, dirs, files in os.walk("raw-img"):
    for name in files:
        filename = os.path.join(root, name)
        with Image.open(filename) as im:
            size = im.size
            if size == target_size:
                continue

            longest = min(size)
            scale = target_size[0]/longest
            
            if size[0] < size[1]:
                resized_image = im.resize((target_size[0], int(size[1] * scale)), resample=Image.Resampling.BICUBIC)
            else:
                resized_image = im.resize((int(size[0] * scale), target_size[0]), resample=Image.Resampling.BICUBIC)

        x1 = (resized_image.size[0] - target_size[0]) // 2
        y1 = (resized_image.size[1] - target_size[1]) // 2
        x2 = x1 + target_size[0]
        y2 = y1 + target_size[1]
        cropped_image = resized_image.crop((x1, y1, x2, y2))

        cropped_image.save(filename)

## Import and prepare dataset

In [52]:
class_names = ("cane", "cavallo", "elefante", "farfalla", "gallina", "gatto", "mucca", "pecora", "ragno", "scoiattolo")
batch_size = 16
train_val_split = 0.2

ds = tf.keras.utils.image_dataset_from_directory(
    "raw-img",
    image_size=target_size,
    batch_size=batch_size)

size = len(ds)
train_size, test_size = int(8/10 * size), int(2/10 * size)
while train_size + test_size < size:
    train_size += 1

train_ds = ds.take(train_size)
val_ds = train_ds.take(int(0.1 * len(train_ds)))
test_ds = ds.skip(train_size).take(test_size)

Found 26179 files belonging to 10 classes.


## Build model and train

In [58]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255),
    # [64, 64, 3]
    tf.keras.layers.Conv2D(filters=32, kernel_size=4, strides=2, activation="relu"),
    # [31, 31, 32]
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, activation="relu"),
    # [29, 29, 64]
    tf.keras.layers.MaxPooling2D(pool_size=3, strides=2),
    # [14, 14, 64]
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, activation="relu"),
    # [12, 12, 64]
    tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, activation="relu"),
    # [10, 10, 64]
    tf.keras.layers.Flatten(),
    # 6400
    #tf.keras.layers.Dense(units=6400, activation="relu"),
    #tf.keras.layers.Dropout(0.5),
    #tf.keras.layers.Dense(units=1024, activation="relu"),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10, activation="softmax")
])

#cnn.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
cnn.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

cnn.build((None, target_size[0], target_size[1], 3))
cnn.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_17 (Rescaling)    (None, 64, 64, 3)         0         
                                                                 
 conv2d_60 (Conv2D)          (None, 31, 31, 32)        1568      
                                                                 
 conv2d_61 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 14, 14, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_62 (Conv2D)          (None, 12, 12, 64)        36928     
                                                                 
 conv2d_63 (Conv2D)          (None, 10, 10, 128)       73856     
                                                     

In [59]:
history = cnn.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1310/1310 [==============================] - 11s 8ms/step - loss: 2.1268 - accuracy: 0.2357 - val_loss: 1.9437 - val_accuracy: 0.3235
Epoch 2/30
1310/1310 [==============================] - 11s 8ms/step - loss: 1.8483 - accuracy: 0.3591 - val_loss: 1.5964 - val_accuracy: 0.4628
Epoch 3/30
1310/1310 [==============================] - 11s 9ms/step - loss: 1.6517 - accuracy: 0.4318 - val_loss: 1.5105 - val_accuracy: 0.4976
Epoch 4/30
1310/1310 [==============================] - 11s 8ms/step - loss: 1.5368 - accuracy: 0.4733 - val_loss: 1.3174 - val_accuracy: 0.5463
Epoch 5/30
1310/1310 [==============================] - 11s 8ms/step - loss: 1.4462 - accuracy: 0.5067 - val_loss: 1.2726 - val_accuracy: 0.5816
Epoch 6/30
1310/1310 [==============================] - 11s 8ms/step - loss: 1.3650 - accuracy: 0.5316 - val_loss: 1.1380 - val_accuracy: 0.6059
Epoch 7/30
1310/1310 [==============================] - 11s 8ms/step - loss: 1.2774 - accuracy: 0.5631 - val_loss: 1.0810 - val_ac

## Evaluate model

In [60]:
score = cnn.evaluate(test_ds)

327/327 [==============================] - 6s 6ms/step - loss: 2.1406 - accuracy: 0.6066
